In [1]:
from utils import *
import pandas as pd

In [2]:
orig_file = "/home/pascalgrosset/data/cosmo/vel_analysis/orig-499.haloproperties"
comp_file = "/home/pascalgrosset/data/cosmo/vel_analysis/sz_PosNoComp_vel_.01-499.haloproperties"

In [3]:
variables_to_show = ["fof_halo_count"
,"fof_halo_tag"
,"fof_halo_mass"
,"fof_halo_ke"
,"fof_halo_center_x"
,"fof_halo_center_y"
,"fof_halo_center_z"
,"fof_halo_angmom_x"
,"fof_halo_angmom_y"
,"fof_halo_angmom_z"
,"fof_halo_max_cir_vel"
,"fof_halo_com_x"
,"fof_halo_com_y"
,"fof_halo_com_z"
,"fof_halo_mean_vx"
,"fof_halo_mean_vy"
,"fof_halo_mean_vz"
,"fof_halo_vel_disp"
,"fof_halo_1D_vel_disp"]

In [4]:
# Read in the data
data_orig_gio = pygio.read_genericio(orig_file, variables_to_show)
data_comp_gio = pygio.read_genericio(comp_file, variables_to_show)

# Read in the mathched list of halos created from matchHalosPos.py
index_file = read_csv_to_array('/home/pascalgrosset/projects/VizAly-Vis_IO/results/indices_com_sz_PosNoComp_vel_.01-499.csv',',')

In [5]:
def adrianVecDiffMag(v1, v2):
    '''Comparison is with Adrian comparison metric:  ||a2 - a1|| / ||a1||: a1=uncompressed, a2=compressed'''
    return ( np.linalg.norm(v2-v1)/ np.linalg.norm(v1) )


def computeMaxDiffMag(indexFile, data_orig_gio, data_comp_gio, numItems):
    '''Create a list of tuples for ang mom diff'''
    data_orig__ang_mom_x = np.array(data_orig_gio["fof_halo_angmom_x"])
    data_orig__ang_mom_y = np.array(data_orig_gio["fof_halo_angmom_y"])
    data_orig__ang_mom_z = np.array(data_orig_gio["fof_halo_angmom_z"])
    
    data_comp__ang_mom_x = np.array(data_comp_gio["fof_halo_angmom_x"])
    data_comp__ang_mom_y = np.array(data_comp_gio["fof_halo_angmom_y"])
    data_comp__ang_mom_z = np.array(data_comp_gio["fof_halo_angmom_z"])
    
    diffMagList = []

    for i in range(numItems):
        index_comp = i
        index_orig = int(indexFile[index_comp])
        
        vec_orig = np.array([data_orig__ang_mom_x[index_orig], data_orig__ang_mom_y[index_orig], data_orig__ang_mom_z[index_orig]])
        vec_comp = np.array([data_comp__ang_mom_x[index_comp], data_comp__ang_mom_y[index_comp], data_comp__ang_mom_z[index_comp]])
    
        magDiff = adrianVecDiffMag(vec_orig, vec_comp)
#         if i == 0:
#             print(vec_orig, vec_comp, magDiff)
#             print(index_orig, index_comp)
        
        diffMagList.append( (i,magDiff) )
        
    return diffMagList



def sortTuple(tup, rev=True): 
    '''Sort a list of tuples in descending order'''
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    return(sorted(tup, key = lambda x: x[1], reverse=rev))



def getFilteredList(data, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, orig=True):
    '''Filter the list based on > filter val'''
    fullList_orig=[]
    count = 0
    
    for i in range(num_elem):
        _index_to_show = sortedList[i][0]
        if orig == True:
            __index_to_show = int(index_file[_index_to_show])
        else:
            __index_to_show = _index_to_show
        
        
        if data[filter_var_name][__index_to_show] > filter_var_value:
            #print(data_show_orig_gio[filter_var_name][__index_to_show])
            
            theList = []
            theList.append(sortedList[i][1])
            for var in variables_to_show:
                theList.append(data[var][__index_to_show])

            fullList_orig.append(theList)

            count = count + 1
            if num_to_show != -1:
                if count > num_to_show:
                    break
    
    return fullList_orig

In [6]:
# Find the halos with the largest difference in ang mom diff
theList = computeMaxDiffMag(index_file, data_orig_gio, data_comp_gio, len(index_file))

# Sort the list in desc order of difference in ang mom diff
sortedList = sortTuple(theList)

In [7]:
num_to_show = 100
num_elem = len(index_file)

filter_var_name = "fof_halo_count"
filter_var_value = 0


# Use above list to get a list of halos sorted by ang mom diff and filtered by size of halos
_fullList_orig = getFilteredList(data_orig_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, True)
_fullList_comp = getFilteredList(data_comp_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, False)


# Create data frames for easier manipulation
cols=['mag_diff','fof_halo_count', 'fof_halo_tag', 'fof_halo_mass', 'fof_halo_ke', 'fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z', 'fof_halo_angmom_x', 'fof_halo_angmom_y', 'fof_halo_angmom_z', 'fof_halo_max_cir_vel', 'fof_halo_com_x', 'fof_halo_com_y', 'fof_halo_com_z', 'fof_halo_mean_vx', 'fof_halo_mean_vy', 'fof_halo_mean_vz', 'fof_halo_vel_disp', 'fof_halo_1D_vel_disp']

_df_orig = pd.DataFrame(_fullList_orig, columns=cols)
_df_comp = pd.DataFrame(_fullList_comp, columns=cols)

_df_orig.to_csv("results/largest_100_ang_mom_diff__orig-499.csv")
_df_comp.to_csv("results/largest_100_ang_mom_diff__PosNoComp_vel_.01-499.csv")

In [8]:
num_to_show = -1
num_elem = len(index_file)

filter_var_name = "fof_halo_count"
filter_var_value = 10000


# Use above list to get a list of halos sorted by ang mom diff and filtered by size of halos
_fullList_orig = getFilteredList(data_orig_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, True)
_fullList_comp = getFilteredList(data_comp_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, False)


# Create data frames for easier manipulation
cols=['mag_diff','fof_halo_count', 'fof_halo_tag', 'fof_halo_mass', 'fof_halo_ke', 'fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z', 'fof_halo_angmom_x', 'fof_halo_angmom_y', 'fof_halo_angmom_z', 'fof_halo_max_cir_vel', 'fof_halo_com_x', 'fof_halo_com_y', 'fof_halo_com_z', 'fof_halo_mean_vx', 'fof_halo_mean_vy', 'fof_halo_mean_vz', 'fof_halo_vel_disp', 'fof_halo_1D_vel_disp']

_df_orig = pd.DataFrame(_fullList_orig, columns=cols)
_df_comp = pd.DataFrame(_fullList_comp, columns=cols)

_df_orig.to_csv("results/largest_above_10000_ang_mom_diff__orig-499.csv")
_df_comp.to_csv("results/largest_above_10000_ang_mom_diff__PosNoComp_vel_.01-499.csv")

In [9]:
_df_orig.head()

,mag_diff,fof_halo_count,fof_halo_tag,fof_halo_mass,fof_halo_ke,fof_halo_center_x,fof_halo_center_y,fof_halo_center_z,fof_halo_angmom_x,fof_halo_angmom_y,fof_halo_angmom_z,fof_halo_max_cir_vel,fof_halo_com_x,fof_halo_com_y,fof_halo_com_z,fof_halo_mean_vx,fof_halo_mean_vy,fof_halo_mean_vz,fof_halo_vel_disp,fof_halo_1D_vel_disp
0,0.494269,13913,944404240,7.633143e+12,1.493101e+18,231.564255,171.483490,187.018188,3.138218e+12,-1.459259e+11,1.263658e+12,291.464355,231.592957,171.456345,187.028275,215.833099,147.782639,-341.153320,262.301392,262.301392
1,0.223249,11572,161068636,6.348791e+12,1.498236e+18,29.268116,147.719147,158.288269,3.638809e+12,1.181091e+13,1.208839e+13,231.398178,29.340719,147.866074,158.213669,-245.429306,-208.106308,463.942566,225.970993,225.970993
2,0.098760,23353,47178029,1.281225e+13,3.431212e+18,16.116560,245.873962,76.782547,4.562504e+13,-7.316554e+12,-4.894625e+13,312.419006,16.104780,245.696564,76.936111,194.033920,-475.092163,69.221329,298.586182,298.586182
3,0.085458,65689,134668705,3.603921e+13,1.449500e+19,39.296761,107.874031,112.195496,2.811799e+14,6.145768e+14,-4.665093e+14,410.259918,39.589939,107.753105,111.828865,170.206512,-329.618011,356.524048,424.136047,424.136047
4,0.079385,24311,337782544,1.333784e+13,2.988704e+18,81.264755,25.631676,197.486725,3.103779e+13,7.988003e+11,-1.349128e+13,369.122131,81.240341,25.603516,197.526672,11.209210,-255.927628,-181.965286,341.281036,341.281036


In [10]:
_df_comp.head()

,mag_diff,fof_halo_count,fof_halo_tag,fof_halo_mass,fof_halo_ke,fof_halo_center_x,fof_halo_center_y,fof_halo_center_z,fof_halo_angmom_x,fof_halo_angmom_y,fof_halo_angmom_z,fof_halo_max_cir_vel,fof_halo_com_x,fof_halo_com_y,fof_halo_com_z,fof_halo_mean_vx,fof_halo_mean_vy,fof_halo_mean_vz,fof_halo_vel_disp,fof_halo_1D_vel_disp
0,0.494269,13913,944404240,7.633143e+12,1.492076e+18,231.564255,171.483490,187.018188,4.275731e+12,1.081207e+12,1.224891e+12,291.464355,231.592957,171.456345,187.028275,215.761200,147.733414,-341.027130,262.214355,262.214355
1,0.223249,11572,161068636,6.348791e+12,1.497231e+18,29.268116,147.719147,158.288269,5.493737e+11,1.217568e+13,9.804108e+12,231.398178,29.340719,147.866074,158.213669,-245.324570,-208.038101,463.808441,225.888199,225.888199
2,0.098760,23353,47178029,1.281225e+13,3.428405e+18,16.116560,245.873962,76.782547,4.684583e+13,-1.163020e+13,-5.385489e+13,312.419006,16.104780,245.696564,76.936111,193.954849,-474.874084,69.196381,298.476227,298.476227
3,0.085458,65689,134668705,3.603921e+13,1.448593e+19,39.296761,107.874031,112.195496,3.361937e+14,6.170240e+14,-4.230048e+14,410.259918,39.589939,107.753105,111.828865,170.139587,-329.510986,356.415985,424.005035,424.005035
4,0.079385,24311,337782544,1.333784e+13,2.986596e+18,81.264755,25.631676,197.486725,3.303451e+13,-9.990961e+11,-1.354393e+13,369.122131,81.240341,25.603516,197.526672,11.212300,-255.832169,-181.912247,341.159882,341.159882
